# Sessie 3 - Stelsels van lineaire vergelijkingen

Voer de onderstaande code uit.

In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np

/opt/anaconda3/lib/python3.5/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
/opt/anaconda3/lib/python3.5/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


## Protein folding en PSICOV

De Protein Data Bank ([PDB](http://www.rcsb.org/pdb/home/home.do)) is een database die allerlei informatie bevat over proteinen. Deze database bevat o.a. de 3D structuur van proteinen. Om deze structuur te bepalen (zie [protein folding](https://en.wikipedia.org/wiki/Protein_folding)), hebben we geautomatiseerde methodes nodig.

We kunnen bijvoorbeeld kijken naar hoe proteinen evolueren. Het is geweten dat de 3D proteinstructuur bij mutatie meestal wordt behouden, en dus de amino-zuren (i.e., de deeltjes die een protein opmaken) gebonden moeten blijven. Dit wil zeggen dat de aminozuren die in contact zijn met elkaar moeten co-muteren om de chemische bindingen te behouden. Met andere woorden, als we naar <i>verschillende</i> sequenties van <i>hetzelfde</i> protein kijken, kunnen we relaties vinden tussen aminozuren in termen van co-mutaties.

Een statistische methode dat gebruik maakt van dit idee, heet [PSICOV](http://www.ncbi.nlm.nih.gov/pubmed/22101153) en werkt als volgt:
1. Stel een covariantie matrix van amino-paren op uit protein data.
2. Inverteer deze matrix om correlaties te bekomen tussen amino-paren
3. Bereken voor ieder paar wat de waarschijnlijkheid is dat deze zuren in contact zijn.

De code voor stap 1 en 3 is gegeven, maar het cruciale gedeelte van deze methode is stap 2, namelijk het inverteren van de covariantie matrix, daar we tot nu toe niet echt stabiele of efficiente methodes hebben gezien. Stabiliteit is nodig, aangezien we zullen werken met een $420 \times 420$ matrix voor een kleine sequentie van aminozuren (in de praktijk is dit gemiddeld een $3150 \times 3150$ matrix).

## 1. Cholesky Decomposition

Toon aan dat de $LU$ ontbinding voor een symmetrische positief semi-definiete matrix $A$ (i.e., $\forall z \neq 0: z'Az \ge 0$) zich herleidt tot een $LL'$ ontbinding en geef de formules om de elementen van $L$ te berekenen. Deze ontbinding heet de Cholesky decompositie. Implementeer deze methode.

In [ ]:
def calclii(A, i):
    for i in range 

def cholesky(A):
    """
    Bereken de Cholesky decompositie A = LL' en geef de matrix L terug.
    """
    n = len(A)
    L = np.zeros((n, n))
    ### FILL IN THE ENTRIES OF L ###

In [ ]:
### TEST ###
L = np.asmatrix([[1, 0, 0],[2, 3, 0],[4, 5, 6]])
A = L*L.transpose()
L_new = cholesky(A)
print("Error: ", str(np.linalg.norm(L_new - L)))

## 2. Stabiliteit

Een LU-decompositie is numeriek niet sterk stabiel.
Neem $F(L, U) = A + \Delta A$.<br>
Dan wordt de onvermijdelijke fout gegeven door:
$$\delta F \le \frac{||L|| \cdot ||U||}{||A||} \cdot C \cdot \epsilon$$
met $\epsilon$ de machineprecisie en $C$ een constante.<br>
Aangezien $||L||$ en $||U||$ arbitrair groot kunnen worden, is dit algoritme niet sterk stabiel.

Leg uit waarom een Cholesky decompositie <i>wel</i> sterk stabiel is.

(Opmerking: $||M|| = \sqrt{\sum\limits_{i,j} M_{ij}^2}$ voor een matrix $M$.)

## 3. Inverse covariantie matrix

Aangezien iedere covariantie matrix een positief semi-definiete matrix is, kunnen we de Cholesky decompositie van een covariantie matrix op een stabiele manier berekenen.

We kunnen dit toepassen op de covariantie matrix van het protein <i>1ag6A</i> om zo de PSICOV methode te vervolledigen.

De volgende files worden in de test-code gebruikt en mogen niet aangepast worden:
* $\texttt{1ag6A-20.csv}$ - Covariantie matrix
* $\texttt{1ag6A.CB}$ - Werkelijke contacten
* $\texttt{psicov.py}$ - Code om de covariantie matrix in te lezen en contacten te voorspellen (stap 1 en 3 van PSICOV)

Kan je gebruikmakend van jouw geimplementeerde Cholesky decompositie de inverse berekenen en de PSICOV methode vervolledigen?

In [ ]:
import scipy.linalg

#Functies om Ax = b op te lossen, gebruikmakend van voorwaartse en
#achterwaartste substitutie met parameters A en b.
forward = lambda A, b: scipy.linalg.solve_triangular(A, b, lower=True)
backward = lambda A, b: scipy.linalg.solve_triangular(A, b, lower=False)
    
def inv_cov(M):
    """
    Inverteer een covariantie matrix M.
    """
    pass ### CODE HERE ###

In [ ]:
%run psicov.py

### TEST ###

#Retrieve the covariance matrix of 1ag6A and compute the inverse
M = retrieve_covariance_matrix('1ag6A-20.csv')
M_inv = inv_cov(M)

#Predict the amino-amino contacts and compare it with the actual contacts
pred_contacts = retrieve_predicted_contacts(M_inv)
true_contacts = retrieve_true_contacts('1ag6A.CB', 8)
n = range(1, 150)
accuracies = [fraction_of_correct_predictions(pred_contacts[:i], true_contacts) for i in n] #Compute the

### PLOT ###
plt.plot(n, accuracies)
plt.xlabel('Top n predicted contacts')
plt.ylabel('Accuracy')
plt.xlim(1,150)
plt.ylim(0,1)
plt.show()